<a href="https://colab.research.google.com/github/aheiX/Teaching/blob/main/TSP_Heuristiken.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TSP - Heuristiken

## Daten

In [5]:
!pip install haversine
!pip install pandas
!pip install plotly.express

import haversine
import pandas as pd
import plotly.express as px

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Daten erstellen
input_data = dict()
input_data["Halle (Saale)"]=dict(id=1, lon=11.9672201241301, lat=51.4970124613279, pop=238.061)
input_data["Magdeburg"]=dict(id=2, lon=11.6363317198121, lat=52.1229002894915, pop=236.188)
input_data["Dessau-Roßlau"]=dict(id=3, lon=12.2331141970666, lat=51.8442730013004, pop=78.731)
input_data["Lutherstadt Wittenberg"]=dict(id=4, lon=12.6357720180488, lat=51.8737704942318, pop=44.984)
input_data["Weißenfels"]=dict(id=5, lon=11.9683230903702, lat=51.1983573259, pop=39.745)
input_data["Halberstadt"]=dict(id=6, lon=11.0495250643003, lat=51.8912329116255, pop=38.682)
input_data["Stendal"]=dict(id=7, lon=11.8508504963119, lat=52.602278283988, pop=38.359)

# Daten als Dataframe
df_input = {col_name: [input_data[name][col_name] for name in input_data] for col_name in ['id', 'lon', 'lat', 'pop']}
df_input['name'] = input_data.keys()
df_input = pd.DataFrame(data=df_input, columns=['name', 'id', 'pop', 'lon', 'lat'])

# print(df)
print(df_input.to_latex(index=False))

# Distanzen berechnen
dist = {i: {j: int(haversine.haversine((input_data[i]['lat'], input_data[i]['lon']), (input_data[j]['lat'], input_data[j]['lon'])))
            for j in input_data}
        for i in input_data}

# Distanzen als Dataframe
dist_df = {input_data[destin]['id']: [dist[origin][destin] for origin in input_data.keys()] for destin in input_data.keys()}
dist_df['origin'] = input_data.keys()
dist_df = pd.DataFrame(data=dist_df,
                       columns=['origin'] + [i+1 for i in range(len(input_data.keys()))]
                       )

# print(dist_df)
print(dist_df.to_latex(index=False))


<ipython-input-3-3a88e59294be>:17: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_input.to_latex(index=False))


\begin{tabular}{lrrrr}
\toprule
                  name &  id &     pop &       lon &       lat \\
\midrule
         Halle (Saale) &   1 & 238.061 & 11.967220 & 51.497012 \\
             Magdeburg &   2 & 236.188 & 11.636332 & 52.122900 \\
         Dessau-Roßlau &   3 &  78.731 & 12.233114 & 51.844273 \\
Lutherstadt Wittenberg &   4 &  44.984 & 12.635772 & 51.873770 \\
            Weißenfels &   5 &  39.745 & 11.968323 & 51.198357 \\
           Halberstadt &   6 &  38.682 & 11.049525 & 51.891233 \\
               Stendal &   7 &  38.359 & 11.850850 & 52.602278 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrrrr}
\toprule
                origin &   1 &   2 &  3 &   4 &   5 &   6 &   7 \\
\midrule
         Halle (Saale) &   0 &  73 & 42 &  62 &  33 &  76 & 123 \\
             Magdeburg &  73 &   0 & 51 &  73 & 105 &  47 &  55 \\
         Dessau-Roßlau &  42 &  51 &  0 &  27 &  74 &  81 &  88 \\
Lutherstadt Wittenberg &  62 &  73 & 27 &   0 &  88 & 108 &  97 \\
            Weißenfels & 

<ipython-input-3-3a88e59294be>:32: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(dist_df.to_latex(index=False))


## Nächster Nachbar Heuristik

In [22]:
heuristik = 'Nächster Nachbar'
heuristik = 'Cheapest Insertion'

tour_info = {'start': [], 'distance': []}

data_map = {'name': [], 'id': [], 'lon': [], 'lat': [], 'start': []}

for depot in input_data.keys():
  tour = [depot]
  total_distance = 0

  if heuristik == 'Nächster Nachbar':
    while len(tour) < len(input_data.keys()):
      next_node = None
      for destin in input_data.keys():
        if destin not in tour:
          if next_node is None or dist[tour[-1]][next_node] > dist[tour[-1]][destin]:
            next_node = destin

      total_distance += dist[tour[-1]][next_node]
      tour.append(next_node)

    total_distance += dist[tour[-1]][depot]
    tour.append(depot)

  elif heuristik == 'Cheapest Insertion':
    min_dist = None
    for origin in input_data.keys():
      for destin in input_data.keys():
        if origin != destin:
          if min_dist is None or dist[origin][destin] < min_dist:
            tour = [origin, destin]

    print(tour)
    while len(tour) < len(input_data.keys()):
      next_tour = None
      min_detour = None

      for node in input_data.keys():
        if node not in tour:
          # print(node)
          for pos in range(len(tour)-1):
            # print(pos)
            detour = dist[tour[pos]][node] + dist[node][tour[pos+1]] - dist[tour[pos]][tour[pos-1]]

            if next_tour is None or min_detour > detour:
              next_tour = tour[:pos+1] + [node] + tour[pos+1:]
              min_detour = detour
              # print(next_tour)

    tour = next_tour
    total_distance = sum(dist[tour[pos]][tour[pos+1]] for pos in range(len(tour)-1))

    # total_distance += dist[tour[-1]][depot]
    # tour.append(depot)



  # Info für Tabelle hinzufügen
  tour_info['start'].append(depot)
  tour_info['distance'].append(total_distance)
  # print('Start: ' + depot + r'\\')
  # print('Tour: ' + str(tour) + r'\\')
  # print('Distanz (in km): ' + str(total_distance) + r'\\')
  # print('')

  # Dataframe für Plot hinzufügen
  for n in tour:
    data_map['name'].append(n)
    data_map['id'].append(input_data[n]['id'])
    data_map['lon'].append(input_data[n]['lon'])
    data_map['lat'].append(input_data[n]['lat'])
    data_map['start'].append('[' + str(input_data[depot]['id']) + '] ' + str(depot) + ', dist: ' + str(total_distance) + ' km')


# Tabelle
df_tab = pd.DataFrame(data=tour_info)
print(df_tab.to_latex(index=False))

# Plot
df_map = pd.DataFrame(data=data_map)
fig = px.line_geo(df_map, lon='lon', lat='lat', text='id',
                  fitbounds='locations',
                  hover_name='name',
                  facet_col='start',
                  facet_col_wrap=2,
                  height=4*200,
                  width=2*400
                  )
fig.update_traces(
    marker=dict(size=10),
    textposition='bottom center' # ['top left', 'top center', 'top right', 'middle left', 'middle center', 'middle right', 'bottom left', 'bottom center', 'bottom right']
)
fig.show()

['Stendal', 'Halberstadt']


KeyboardInterrupt: ignored